In [ ]:
# Install necessary packages
# !pip install PyMuPDF markdown python-dotenv --quiet

import gradio as gr
import requests
import json
import fitz  # PyMuPDF
from markdown import markdown
import os
from dotenv import load_dotenv

# --- Load environment variables from .env ---
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# --- Configuration ---
API_URL = "https://api.groq.com/openai/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}
MODEL_NAME = "llama3-70b-8192"

# --- Global Storage ---
stored_legal_text = ""
simplified_summary = ""

# --- Prompts for Different Simplification Levels ---
PROMPTS = {
    "Quick Summary (ELI5)": """...""",  # (Use your original long strings here)
    "Standard View": """...""",
    "Detailed Breakdown": """..."""
}

# Simplify logic
def simplify_text(text, level):
    global simplified_summary
    prompt = PROMPTS[level].format(text=text)
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a world-class legal document simplifier. You follow instructions precisely."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2, "max_tokens": 2000
    }
    try:
        response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return f"⚠️ API Error: {e}"
    raw_output = response.json()["choices"][0]["message"]["content"].strip()
    simplified_summary = raw_output
    return markdown(raw_output)

# PDF extraction
def extract_text_from_pdf(file_obj):
    try:
        with fitz.open(file_obj.name) as doc:
            return "\n".join([page.get_text() for page in doc])
    except Exception as e:
        return f"⚠️ PDF Error: {e}"

# Input handler
def handle_input(input_mode, file_input, text_input, simplicity_level):
    global stored_legal_text, simplified_summary
    simplified_summary = ""
    if input_mode == "Upload PDF":
        if file_input is None: return "⚠️ Please upload a PDF file."
        text = extract_text_from_pdf(file_input)
    elif input_mode == "Paste Text":
        if not text_input.strip(): return "⚠️ Please paste some legal text."
        text = text_input
    else:
        return "⚠️ Invalid input mode selected."
    if not text.strip(): return "⚠️ No text found to simplify."
    stored_legal_text = text.strip()
    return simplify_text(stored_legal_text, simplicity_level)

# Chatbot function
def legal_chatbot(user_input, history):
    global simplified_summary
    if not simplified_summary.strip():
        return "⚠️ Please simplify a legal document first before asking questions."
    system_prompt = f"""
    You are a helpful legal assistant. A user has received a simplified summary of a legal document.
    Your job is to answer their questions based ONLY on the provided summary.
    Be friendly, clear, and concise.

    Summary:
    ---
    {simplified_summary}
    ---
    Now answer the question.
    """
    messages = [{"role": "system", "content": system_prompt}]
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    messages.append({"role": "user", "content": user_input})
    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": 0.5,
        "max_tokens": 1000
    }
    try:
        response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"].strip()
    except requests.exceptions.RequestException as e:
        return f"⚠️ API Error: {e}"

# 🌐 Gradio UI
with gr.Blocks(theme=gr.themes.Soft(), css="#disclaimer {color: grey; font-size: 0.9em;}") as demo:
    gr.Markdown("## Legal Document Simplifier + Chat Assistant")
    with gr.Row():
        with gr.Column(scale=2):
            input_mode = gr.Radio(["Upload PDF", "Paste Text"], label="Input Method", value="Upload PDF")
            pdf_input = gr.File(label="Upload PDF File", file_types=[".pdf"], visible=True)
            text_input = gr.Textbox(label="Paste Legal Text Here", lines=12, visible=False)
            input_mode.change(
                lambda mode: (gr.update(visible=mode == "Upload PDF"), gr.update(visible=mode == "Paste Text")),
                inputs=input_mode,
                outputs=[pdf_input, text_input]
            )
            simplicity_level = gr.Radio(["Quick Summary (ELI5)", "Standard View", "Detailed Breakdown"], label="Simplification Level", value="Standard View")
            simplify_btn = gr.Button("Simplify Document")
            simplified_output = gr.HTML()
        with gr.Column(scale=1):
            chatbot = gr.ChatInterface(fn=legal_chatbot)
    simplify_btn.click(fn=handle_input, inputs=[input_mode, pdf_input, text_input, simplicity_level], outputs=[simplified_output])
    gr.Markdown("_Disclaimer: This tool is for informational purposes only._", elem_id="disclaimer")

demo.launch(debug=True, share=True)


/tmp/ipython-input-2-1736023006.py:179: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(label="Chat about your document", height=500, bubble_full_width=False, avatar_images=(None, "https://upload.wikimedia.org/wikipedia/commons/a/a4/G-logo.svg")),
/tmp/ipython-input-2-1736023006.py:179: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot=gr.Chatbot(label="Chat about your document", height=500, bubble_full_width=False, avatar_images=(None, "https://upload.wikimedia.org/wikipedia/commons/a/a4/G-logo.svg")),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:322: UserWarning: The gr.ChatInterface

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://baa4238833d2364290.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
